In [ ]:
#@title 1. Install Dependencies
#@markdown This cell installs the necessary libraries from Unsloth and Hugging Face.
# %%capture
import os

!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
!pip install --no-deps unsloth

# Install latest transformers for Gemma 3N
!pip install --no-deps git+https://github.com/huggingface/transformers.git
!pip install --no-deps --upgrade timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.7/162.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.7.2 

In [ ]:
#@title 2. Load Model and Tokenizer
#@markdown This cell loads the pre-trained Gemma 3N model with 4-bit quantization for efficient inference.
from unsloth import FastModel
import torch

# Load the model and tokenizer from Unsloth's Hugging Face repository
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3n-E4B-it-unsloth-bnb-4bit", # A 4-bit quantized model
    dtype = None,                 # None for auto detection
    max_seq_length = 2048,        # Max sequence length for context
    load_in_4bit = True,          # 4bit for efficiency
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.7.2: Fast Gemma3N patching. Transformers: 4.54.0.dev0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3N does not support SDPA - switching to eager!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.72G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.15G [00:00<?, ?B/s]

...

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

In [ ]:
#@title 3. Define the Inference Function
#@markdown This helper function will handle the process of generating a response from the model.
#@markdown It takes the formatted prompt, generates text, and returns the clean output.

from transformers import TextStreamer

def run_inference(model, tokenizer, messages, max_new_tokens=512):
    """
    Runs inference on the model and returns the decoded text output.
    """
    # Apply the chat template and tokenize
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt = True, # for generation
        tokenize = True,
        return_tensors = "pt",
        return_dict = True,
    ).to("cuda")

    # Generate the output
    outputs = model.generate(
        **inputs,
        max_new_tokens = max_new_tokens,
        temperature = 1.0,
        top_p = 0.95,
        top_k = 64,
        use_cache = True,
    )

    # Decode and clean the output
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # Clean the prompt from the output string to get only the generation
    prompt_length = len(tokenizer.batch_decode(inputs['input_ids'], skip_special_tokens=True)[0])
    clean_output = decoded_output[prompt_length:].strip()

    return clean_output

In [ ]:
#@title 4. Perform Zero-Shot Evaluation

# ==========================================================================================
# == USER INPUT: the path to meme image and the text content of the meme. ==
# ==========================================================================================
meme_image_path = "/content/1.png" #@param {type:"string"}
meme_text = "Atlantic Ocean| EUROPE Mediterranean Sea AFRICA • Encyclopaedia Britannica, Inc. What separates humans from animals? Black Sea ASIA" #@param {type:"string"}
# ==========================================================================================


# STEP 1: Analyze the meme with the multi-modal prompt (Image + Text)
print(" STEP 1: Analyzing the meme...")

analysis_prompt = f"""Analyze this meme by considering
both the visual elements and the text content:
"{meme_text}"
Focus on:
1. What is shown in the image
2. How the text relates to the image
3. The overall message or intent
4. Whether this combination promotes bias,
discrimination, or harmful stereotypes
Provide your analysis in a brief paragraph."""

# Formatting the multi-modal input for the model
analysis_messages = [{
    "role" : "user",
    "content": [
        { "type": "image", "image" : meme_image_path },
        { "type": "text",  "text" : analysis_prompt }
    ]
}]

# Getting the model's analysis
analysis = run_inference(model, tokenizer, analysis_messages)
print("\n MODEL ANALYSIS:\n", analysis)


# STEP 2: Make a decision based on the analysis (Text-only)
print("\n STEP 2: Determining if the content is hateful...")

decision_prompt = f"""Based on the following analysis
of a meme:
{analysis}
Determine if this meme contains hateful content by
considering:
- Does it promote discrimination or prejudice?
- Does it target specific groups based on protected
characteristics?
- Does it use harmful stereotypes or derogatory
language?
- Is the combination of image and text intended to
cause harm or offense?
Respond with ONLY'hateful' or 'not hateful', followed
by a brief explanation after a semicolon."""

# Formating the text input for the model
decision_messages = [{
    "role" : "user",
    "content": [{
        "type" : "text",
        "text" : decision_prompt
    }]
}]

# Getting the final decision from the model
decision = run_inference(model, tokenizer, decision_messages, max_new_tokens=32)
print("\n FINAL DECISION:\n", decision)

✅ STEP 1: Analyzing the meme...

📝 MODEL ANALYSIS:
 The image displays a map of the Mediterranean region, showing continents, oceans, and countries. The text "What separates humans from animals?" is superimposed on this geographical context. The juxtaposition of a map depicting human-defined geographical boundaries with a question about the difference between humans and animals is unsettling. It implicitly suggests a separation or distinction between humans and the animal kingdom, potentially reinforcing a human-centric view of the world. While not overtly biased, the combination of visual and textual elements could subtly promote a hierarchical view where humans are seen as fundamentally different from and superior to animals.

✅ STEP 2: Determining if the content is hateful...

🏁 FINAL DECISION:
 not hateful; While the meme's implications are unsettling and potentially promote a human-centric view, it doesn't directly target specific groups, use harmful stereotypes


In [ ]:
#@title 4. Perform Zero-Shot Evaluation

# ==========================================================================================
# == USER INPUT: the path to meme image and the text content of the meme. ==
# ==========================================================================================
meme_image_path = "/content/1.png" #@param {type:"string"}
meme_text = "Atlantic Ocean| EUROPE Mediterranean Sea AFRICA • Encyclopaedia Britannica, Inc. What separates humans from animals? Black Sea ASIA" #@param {type:"string"}
# ==========================================================================================


# STEP 1: Analyze the meme with the multi-modal prompt (Image + Text)
print(" STEP 1: Analyzing the meme...")

analysis_prompt = f"""Analyze this meme by considering
both the visual elements and the text content:
"{meme_text}"
Focus on:
1. What is shown in the image
2. How the text relates to the image
3. The overall message or intent
4. Whether this combination promotes bias,
discrimination, or harmful stereotypes
Provide your analysis in a brief paragraph."""

# Formatting the multi-modal input for the model
analysis_messages = [{
    "role" : "user",
    "content": [
        { "type": "image", "image" : meme_image_path },
        { "type": "text",  "text" : analysis_prompt }
    ]
}]

# Getting the model's analysis
analysis = run_inference(model, tokenizer, analysis_messages)
print("\n MODEL ANALYSIS:\n", analysis)


# STEP 2: Make a decision based on the analysis (Text-only)
print("\n STEP 2: Determining if the content is hateful...")

decision_prompt = f"""Based on the following analysis
of a meme:
{analysis}
Determine if this meme contains hateful content by
considering:
- Does it promote discrimination or prejudice?
- Does it target specific groups based on protected
characteristics?
- Does it use harmful stereotypes or derogatory
language?
- Is the combination of image and text intended to
cause harm or offense?
Respond with ONLY'hateful' or 'not hateful', followed
by a brief explanation after a semicolon."""

# Formating the text input for the model
decision_messages = [{
    "role" : "user",
    "content": [{
        "type" : "text",
        "text" : decision_prompt
    }]
}]

# Getting the final decision from the model
decision = run_inference(model, tokenizer, decision_messages, max_new_tokens=32)
print("\n FINAL DECISION:\n", decision)

✅ STEP 1: Analyzing the meme...

📝 MODEL ANALYSIS:
 ## Analysis of the Meme

The image shows a map of the Mediterranean Sea, highlighting the geographical division between Europe and Africa, and also showing the proximity of Asia. The text "What separates humans from animals?" juxtaposed with this geographical map is inherently misleading and promotes a harmful stereotype. It attempts to link the concept of "humans" with a specific geographical region (Europe) and implicitly suggests a separation or superiority over "animals," which are not visually represented. This combination doesn't offer a meaningful answer to the question but rather reinforces a potentially biased and discriminatory view that places humans in a distinct and potentially superior category compared to the animal kingdom. The intent is likely to provoke thought or perhaps humor, but it falls short by relying on a geographically narrow and ultimately flawed premise to address a complex philosophical question.

✅ STEP 